In [1]:
import json
import urllib2
url = "https://raw.githubusercontent.com/suneman/socialgraphs2016/master/files/test.json"
response = urllib2.urlopen(url)
listObj = json.load(response)   

In [2]:
import pandas as pd

In [2]:
def buildWikiQuery(page, last_revision_year):
    # Build query
    baseurl = "http://en.wikipedia.org/w/api.php/?"
    action = "action=query"
    title = "titles=%s" % page 
    content = "prop=revisions"
    rvprop ="rvprop=timestamp|content"
    dataformat = "format=json"
    rvdir = "rvdir=older" #sort revisions: old -> new
    start = "rvend=2000-01-03T00:00:00Z" #start of my time period
    end = "rvstart=%s-01-03T00:00:00Z" % last_revision_year #end of my time period
    limit = "rvlimit=1" #consider only the first revision

    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    return query

In [3]:
buildWikiQuery("PAGE", "2019")

'http://en.wikipedia.org/w/api.php/?action=query&titles=PAGE&prop=revisions&rvprop=timestamp|content&format=json&rvdir=older&rvstart=2019-01-03T00:00:00Z&rvend=2000-01-03T00:00:00Z&rvlimit=1'

In [25]:
def fetchContent(query):
    # Send query to wikipedia
    result = 'No content'
    revision_key = 'revisions'
    response = urllib2.urlopen(query)
    json_reponse = json.load(response)
    pages = json_reponse["query"]['pages']
    page_key = pages.keys()[0]
    page_content = pages[page_key]
    if revision_key in page_content:
        content = pages[page_key][revision_key]
        result = content[0]["*"].encode("utf8")
    return result

In [54]:
def saveToFile(name, year, content):
    if content == 'No content':
        print 'Empty page for:', name
    else:
        filename = 'congress%s/%s.txt' % (year, name)
        f = open(filename, "a")
        f.write(content)
        f.close()  

In [48]:
h113 = pd.read_csv("dataset/H113.csv")
h114 = pd.read_csv("dataset/H114.csv")
h115 = pd.read_csv("dataset/H115.csv")

In [52]:
def fetch(dataframe, year):
    wikipages = list(set(h113.WikiPageName))
    for i, page_name in enumerate(wikipages):
        wiki_query = buildWikiQuery(page_name, year)
        wiki_content = fetchContent(wiki_query)
        saveToFile(page_name, year, wiki_content)

In [55]:
fetch(h113, '2015')

In [ ]:
fetch(h114, '2017')

In [59]:
fetch(h115, '2019')

In [44]:
import os
dir_name = "./congress/"
directory = os.listdir(dir_name)

In [45]:
for item in directory:
    if item.endswith("2019.txt"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
a2 = 1 - w*(1 )